# Qiskit Gradient Framework

The gradient framework enables the evaluation of quantum gradients as well as functions thereof.
Besides standard first order gradients of expectation values of the form
$$ \langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle $$
<!--- $$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $$

$$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2}, $$
--->

the gradient also supports the evaluation of second order gradients (Hessians), and the Quantum Fisher Information (QFI) of quantum states $|\psi\left(\theta\right)\rangle$.

![gradient_framework.png](attachment:gradient_framework.png)

### Imports

In [ ]:
#General imports
import numpy as np

#Operator Imports
from qiskit.aqua.operators import Z, X, I, StateFn, CircuitStateFn, SummedOp, ListOp
from qiskit.aqua.operators.gradients import Gradient, NaturalGradient, QFI, Hessian

#Circuit imports
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.circuit.library import EfficientSU2, RealAmplitudes

## First Order Gradients


Given a parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$, and observable $\hat{O}\left(\omega\right)=\sum_{i}\omega_i\hat{O}_i$, we want to compute...
1. Gradient of an expectation value w.r.t. a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega} $
2.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $
3.  Gradient of sampling probabilities w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta} $





### Gradients w.r.t. Measurement Operator Parameters
$$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega_i} = \langle\psi\left(\theta\right)|\hat{O}_i\left(\omega\right)|\psi\left(\theta\right)\rangle. $$
### Gradients w.r.t. State Parameters
$$\frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $$ and
$$ \frac{\partial p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|i\rangle\langle i |\psi\left(\theta\right)\rangle}{\partial\theta}.$$

### Gradients w.r.t. Measurement Operator Parameters

First of all, we define a quantum state $|\psi\left(\theta\right)\rangle$ and a Hamiltonian $H$ acting as observable. Then, the state and the Hamiltonian are wrapped into an object defining the expectation value $$ \langle\psi\left(\theta\right)|H|\psi\left(\theta\right)\rangle. $$

In [ ]:
# Instantiate the quantum state
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = (coeff_0*coeff_0*2)*X + coeff_1 * Z

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

# Print the operator corresponding to the expectation value
print(op)

ComposedOp([
  OperatorMeasurement(SummedOp([
    2.0*conjugate(c_0)**2 * X,
    1.0*conjugate(c_1) * Z
  ])),
  CircuitStateFn(
        ┌───┐┌───────┐┌───────┐
  q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├
        └───┘└───────┘└───────┘
  )
])


We construct a list of the parameters for which we aim to evaluate the gradient. In this case, the parameters are given as the coefficients of $H$.
Now, this list and the expectation value operator are used to generate the operator which represents the gradient.

In [9]:
# Define the coefficients w.r.t. we want to compute the gradient
obs_coeffs = [coeff_0, coeff_1]

# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi}

# Convert the operator and the gradient target coefficients into the respective operator
grad = Gradient().convert(operator = op, params = obs_coeffs)

# Print the operator corresponding to the Gradient
print(grad)

ListOp([
  4.0*c_0 * ComposedOp([
    OperatorMeasurement(1.00000000000000 * Z),
    CircuitStateFn(
          ┌───┐┌───────┐┌───────┐┌───┐
    q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├┤ H ├
          └───┘└───────┘└───────┘└───┘
    )
  ]),
  1.00000000000000 * ComposedOp([
    OperatorMeasurement(1.00000000000000 * Z),
    CircuitStateFn(
          ┌───┐┌───────┐┌───────┐
    q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├
          └───┘└───────┘└───────┘
    )
  ])
])


/Users/ouf/Documents/GitHub/qiskit-terra/qiskit/quantum_info/operators/symplectic/pauli.py:185: DeprecationWarning: Initializing Pauli from `Pauli(z=z, x=x)` kwargs is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. Use tuple initialization `Pauli((z, x))` instead.
  base_z, base_x, base_phase = self._from_array_deprecated(z, x)


All that is left to do is to assign values to the pararameters and to evaluate the gradient operators.

In [10]:
# Assign the parameters and evaluate the gradient
grad_result = grad.assign_parameters(value_dict).eval()
print('Gradient ', grad_result)

Gradient  [(1.414213562373095+1.12e-16j), 0j]


### Gradients w.r.t. State Parameters
A gradient w.r.t. a state parameter may be evaluated with different methods. Each method has advantages and disadvantages.

In [11]:
# Define the Hamiltonian with fixed coefficients
H = 0.5 * X - 1 * Z
# Define the parameters w.r.t. we want to compute the gradients
params = [a, b]
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state into an expectation value operator
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
print(op)

ComposedOp([
  OperatorMeasurement(0.5 * X
  - 1.0 * Z),
  CircuitStateFn(
        ┌───┐┌───────┐┌───────┐
  q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├
        └───┘└───────┘└───────┘
  )
])


#### Parameter Shift Gradients
<a id='param_shift_grad'></a>
Given a Hermitian operator $g$ with two unique eigenvalues $\pm r$ which acts as generator for a parameterized quantum gate $$G(\theta)= e^{-i\theta g}.$$
Then, quantum gradients can be computed by using eigenvalue $r$ dependent shifts to parameters. 
All [standard, parameterized Qiskit gates](https://github.com/Qiskit/qiskit-terra/tree/master/qiskit/circuit/library/standard_gates) can be shifted with $\pi/2$, i.e.,
 $$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} = 2 \left(\langle\psi\left(\theta+\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\pi/2\right)\rangle -\partial\langle\psi\left(\theta-\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\pi/2\right)\rangle\right).$$
 Probability gradients are computed equivalently.

In [12]:
# Convert the expectation value into an operator corresponding to the gradient w.r.t. the state parameters using 
# the parameter shift method.
state_grad = Gradient(grad_method='param_shift').convert(operator=op, params=params)
# Print the operator corresponding to the gradient
print(state_grad)
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with parameter shift', state_grad_result)

ListOp([
  SummedOp([
    0.25 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a + 1.5707963267949) ├┤ RX(b) ├┤ H ├
            └───┘└─────────────────────────┘└───────┘└───┘
      )
    ]),
    -0.25 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a - 1.5707963267949) ├┤ RX(b) ├┤ H ├
            └───┘└─────────────────────────┘└───────┘└───┘
      )
    ]),
    -0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a + 1.5707963267949) ├┤ RX(b) ├
            └───┘└─────────────────────────┘└───────┘
      )
    ]),
    0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a - 1.5707963267949) ├┤ RX(b) ├
  

 #### Linear Combination of Unitaries Gradients
<a id='lin_comb_grad'></a>
Unitaries can be written as $U\left(\omega\right) = e^{iM\left(\omega\right)}$, where $M\left(\omega\right)$ denotes a parameterized Hermitian matrix. 
Further, Hermitian matrices can be decomposed into weighted sums of Pauli terms, i.e., $M\left(\omega\right) = \sum_pm_p\left(\omega\right)h_p$ with $m_p\left(\omega\right)\in\mathbb{R}$ and $h_p=\bigotimes\limits_{j=0}^{n-1}\sigma_{j, p}$ for $\sigma_{j, p}\in\left\{I, X, Y, Z\right\}$ acting on the $j^{\text{th}}$ qubit. Thus, the gradients of 
$U_k\left(\omega_k\right)$ are given by
\begin{equation*}
\frac{\partial U_k\left(\omega_k\right)}{\partial\omega_k} = \sum\limits_pi \frac{\partial m_{k,p}\left(\omega_k\right)}{\partial\omega_k}U_k\left(\omega_k\right)h_{k_p}.
\end{equation*}

Combining this observation with a circuit structure presented in [Simulating physical phenomena by quantum networks](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.65.042323) allows us to compute the gradient with the evaluation of a single quantum circuit.

In [13]:
# Convert the expectation value into an operator corresponding to the gradient w.r.t. the state parameter using 
# the linear combination of unitaries method.
state_grad = Gradient(grad_method='lin_comb').convert(operator=op, params=params)
# Print the operator corresponding to the gradient
print(state_grad)
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with the linear combination method', state_grad_result)

ListOp([
  SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(ZZ) * 2.0,
      CircuitStateFn(
             ┌───┐          ┌───────┐┌───────┐┌───┐
       q0_0: ┤ H ├────────■─┤ RZ(a) ├┤ RX(b) ├┤ H ├
             ├───┤┌─────┐ │ └─┬───┬─┘└───────┘└───┘
      q41_0: ┤ H ├┤ SDG ├─■───┤ H ├────────────────
             └───┘└─────┘     └───┘                
      ) * 0.7071067811865476
    ]),
    -1.0 * ComposedOp([
      OperatorMeasurement(ZZ) * 2.0,
      CircuitStateFn(
             ┌───┐          ┌───────┐┌───────┐
       q0_0: ┤ H ├────────■─┤ RZ(a) ├┤ RX(b) ├
             ├───┤┌─────┐ │ └─┬───┬─┘└───────┘
      q42_0: ┤ H ├┤ SDG ├─■───┤ H ├───────────
             └───┘└─────┘     └───┘           
      ) * 0.7071067811865476
    ])
  ]),
  SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(ZZ) * 2.0,
      CircuitStateFn(
             ┌───┐┌───────┐┌───┐┌───────┐┌───┐
       q0_0: ┤ H ├┤ RZ(a) ├┤ X ├┤ RX(b) ├┤ H ├
             ├───┤└┬─────┬┘└─┬─┘└─┬───┬─┘└───┘
   

/Users/ouf/Documents/GitHub/qiskit-aqua/qiskit/aqua/operators/primitive_ops/pauli_op.py:107: DeprecationWarning: `kron` is deprecated and will be removed no earlier than 3 months after the release date of Qiskit Terra 0.17.0. Use `expand` instead, but note this does not change the operator in-place.
  return PauliOp(op_copy.kron(self.primitive), coeff=self.coeff * other.coeff)


 #### Finite Difference Gradients

<a id='fin_diff_grad'></a>

Unlike the other methods, finite difference gradients are numerical estimations rather than analytical values.
This implementation employs a central difference approach with $\epsilon << 1$
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} \approx \frac{1}{2\epsilon} \left(\langle\psi\left(\theta+\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\epsilon\right)\rangle - \partial\langle\psi\left(\theta-\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\epsilon\right)\rangle\right).$
 Probability gradients are computed equivalently.

In [14]:
# Convert the expectation value into an operator corresponding to the gradient w.r.t. the state parameter using 
# the finite difference method.
state_grad = Gradient(grad_method='fin_diff').convert(operator=op, params=params)
# Print the operator corresponding to the gradient
print(state_grad)
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with finite difference', np.real(np.round(state_grad_result, 3)))

ListOp([
  SummedOp([
    250000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a + 1.0e-6) ├┤ RX(b) ├┤ H ├
            └───┘└────────────────┘└───────┘└───┘
      )
    ]),
    -250000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a - 1.0e-6) ├┤ RX(b) ├┤ H ├
            └───┘└────────────────┘└───────┘└───┘
      )
    ]),
    -500000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a + 1.0e-6) ├┤ RX(b) ├
            └───┘└────────────────┘└───────┘
      )
    ]),
    500000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a - 1.0e-6) ├┤ RX(b) ├
            └───┘└────────────────┘└───────┘
      )
    ])
  ]),
  SummedOp([
    

### Natural Gradient

A special type of first order gradient is the natural gradient which has proven itself useful in classical machine learning and is already being studied in the quantum context. This quantity represents a gradient that is 'rescaled' with the inverse [Quantum Fisher Information matrix](#qfi) 
$$ QFI ^{-1} \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}.$$

Instead of inverting the QFI, one can also use a least-square solver with or without regularization to solve

$$ QFI x = \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}.$$

The implementation supports ridge and lasso regularization with automatic search for a good parameter using [L-curve corner search](https://arxiv.org/pdf/1608.04571.pdf) as well as two types of perturbations of the diagonal elements of the QFI.

The natural gradient can be used instead of the standard gradient with any gradient-based optimizer and/or ODE solver.

In [15]:
# Besides the method to compute the circuit gradients resp. QFI, a regularization method can be chosen: 
# `ridge` or `lasso` with automatic parameter search or `perturb_diag_elements` or `perturb_diag` 
# which perturb the diagonal elements of the QFI.

nat_grad = NaturalGradient(grad_method='lin_comb', qfi_method='lin_comb_full', regularization='ridge').convert(
                           operator=op, params=params)

# Assign the parameters and evaluate the gradient
nat_grad_result = nat_grad.assign_parameters(value_dict).eval()
print('Natural gradient computed with linear combination of unitaries', np.round(nat_grad_result, 3))

gradient [-0.354  0.707]
fubini study [[ 0.25  -0.   ]
 [-0.     0.219]]
Natural gradient computed with linear combination of unitaries [-2.174  1.902]


### Generic Gradient Function

Suppose you want to evaluate the gradient of a function 
$$F\left(\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle\right),$$ such as $$-\frac{1}{2^n}\sum_i \log{p_i}$$ with $p_i = \langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle.$  This can be done by specifying a `combo_fn` as well as a `grad_combo_fn` for a `ListOp` as is shown in the example below.

Note: Automatic differentiation of trivial functions such as sums and multiplications do not require the specification of a `grad_combo_fn` as they are directly supported.

In [16]:
np.random.seed(2)

def combo_fn(x):
    amplitudes = x[0].primitive.data
    pdf = np.multiply(amplitudes, np.conj(amplitudes))
    return np.sum(np.log(pdf)) / (-len(amplitudes))

def grad_combo_fn(x):
    amplitudes = x[0].primitive.data
    pdf = np.multiply(amplitudes, np.conj(amplitudes))
    grad = []
    for prob in pdf:
        grad += [-1 / prob]
    return grad

qc = RealAmplitudes(2, reps=1)
grad_op = ListOp([StateFn(qc)], combo_fn=combo_fn, grad_combo_fn=grad_combo_fn)
grad = Gradient(grad_method='lin_comb', regularization='ridge'
                       ).convert(grad_op, qc.ordered_parameters)
value_dict = dict(
    zip(qc.ordered_parameters, np.random.rand(len(qc.ordered_parameters))))

print('Gradient of the negative log-likelihood', grad.assign_parameters(value_dict).eval())

Gradient of the negative log-likelihood [[(-0.16666259195178101+0j)], [(-7.244949701825186+0j)], [(-2.979791753055632+0j)], [(-5.31018607783832+0j)]]


## Hessians (Second Order Gradients)

Four types of second order gradients are supported by the gradient framework.
1. Gradient of an expectation value w.r.t. a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega^2} $
2.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2} $
3.  Gradient of sampling probabilities w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial^2 p_i}{\partial\theta^2} = \frac{\partial^2\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta^2} $
4.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter and a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta\partial\omega} $

### Hessians w.r.t. Measurement Operator Parameters

Again, we define a quantum state $|\psi\left(\theta\right)\rangle$ and a Hamiltonian $H$ acting as observable. Then, the state and the Hamiltonian are wrapped into an object defining the expectation value $$ \langle\psi\left(\theta\right)|H|\psi\left(\theta\right)\rangle. $$

In [17]:
# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = coeff_0*coeff_1*coeff_1*X

# Instantiate the quantum state with two parameters
a = Parameter('a')
b = Parameter('b')

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

Next, we can choose the parameters for which we want to compute second order gradients.
- Given a tuple, the `Hessian` will evaluate the second order gradient for the two parameters. 
- Given a list, the `Hessian` will evaluate the second order gradient for all possible combinations of tuples of these parameters.

After binding parameter values to the parameters, the Hessian can be evaluated.

In [18]:
# Define the coefficient tuple w.r.t. which we want to compute the gradient
hessian_coeffs = (coeff_0, coeff_1)

# Convert the operator and the hessian target coefficients into the respective operator
hessian = Hessian().convert(operator = op, params = [coeff_0, coeff_1])

# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi/4}

# Assign the parameters and evaluate the Hessian w.r.t. the Hamiltonian coefficients
hessian_result = hessian.assign_parameters(value_dict).eval()
print('Hessian \n', np.real(np.array(hessian_result)))

Hessian 
 [[ 0.         -1.41421356]
 [-1.41421356  0.70710678]]


### Hessians w.r.t. State Parameters

In [19]:
params = [a, b]

# Get the operator object representing the Hessian
state_hess = Hessian(hess_method='param_shift').convert(operator=op, params=params)
# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(value_dict).eval()
print('Hessian computed using the parameter shift method\n', np.real(np.array(hessian_result)))

# Get the operator object representing the Hessian
state_hess = Hessian(hess_method='lin_comb').convert(operator=op, params=params)
# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(value_dict).eval()
print('Hessian computed using the linear combination of unitaries method\n', np.real(np.array(hessian_result)))

# Get the operator object representing the Hessian using finite difference
state_hess = Hessian(hess_method='fin_diff').convert(operator=op, params=params)
# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(value_dict).eval()
print('Hessian computed with finite difference\n', np.round(np.real(np.array(hessian_result)), 3))

Hessian computed using the parameter shift method
 [[-3.53553391e-01  6.93889390e-17]
 [ 8.32667268e-17  0.00000000e+00]]
Hessian computed using the linear combination of unitaries method
 [[-0.35355339  0.        ]
 [ 0.          0.        ]]
Hessian computed with finite difference
 [[-0.354  0.   ]
 [ 0.     0.   ]]


## <a name="qfi"></a> QFI
The Quantum Fisher Information is a metric tensor which is representative for the representation capacity of a 
parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$.

The entries of the QFI for a pure state reads

$$
QFI_{kl} = 4 * \text{Re}\left[\langle\partial_k\psi|\partial_l|psi\rangle-\langle\partial_k\psi|\psi\rangle\langle\psi|\partial_l\psi\rangle \right].$$

### Circuit QFIs

The evaluation of the QFI corresponding to a quantum state that is generated by a parameterized quantum circuit can be conducted in different ways.

#### Linear Combination Full QFI
To compute the full QFI, we use a working qubit as well as intercepting controlled gates. See e.g. [Variational ansatz-based quantum simulation of imaginary time evolution ](https://www.nature.com/articles/s41534-019-0187-2).

In [20]:
# Wrap the quantum circuit into a CircuitStateFn
state = CircuitStateFn(primitive=qc, coeff=1.)

# Convert the state and the parameters into the operator object that represents the QFI 
qfi = QFI(qfi_method='lin_comb_full').convert(operator=state, params=params)
# Define the values for which the QFI is to be computed
values_dict = {a: np.pi / 4, b: 0.1}

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('full QFI \n', np.real(np.array(qfi_result)))

full QFI 
 [[ 1.00000000e+00 -4.10121933e-17]
 [-4.10121933e-17  5.00000000e-01]]


#### Block-diagonal and Diagonal Approximation
A block-diagonal resp. diagonal approximation of the QFI can be computed without additional working qubits.
This implementation requires the unrolling into Pauli rotations and unparameterized Gates.

In [21]:
# Convert the state and the parameters into the operator object that represents the QFI 
# and set the approximation to 'block_diagonal'
qfi = QFI('overlap_block_diag').convert(operator=state, params=params)

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('Block-diagonal QFI \n', np.real(np.array(qfi_result)))

# Convert the state and the parameters into the operator object that represents the QFI 
# and set the approximation to 'diagonal'
qfi = QFI('overlap_diag').convert(operator=state, params=params)

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('Diagonal QFI \n', np.real(np.array(qfi_result)))

Block-diagonal QFI 
 [[1.  0. ]
 [0.  0.5]]
Diagonal QFI 
 [[1.  0. ]
 [0.  0.5]]


## Application Examples

### Additional Imports

In [22]:
# Execution Imports
from qiskit import Aer
from qiskit.aqua import QuantumInstance

# Algorithm Imports
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import CG

### VQE with gradient based optimization

The Gradient Module can also be used for a gradient-based `VQE`.
First, the Hamiltonian and wavefunction ansatz are initialized.

In [23]:
from qiskit.aqua.operators import I, X, Z
from qiskit.circuit import QuantumCircuit, ParameterVector
from scipy.optimize import minimize

# Instantiate the system Hamiltonian
h2_hamiltonian = -1.05 * (I ^ I) + 0.39 * (I ^ Z) - 0.39 * (Z ^ I) - 0.01 * (Z ^ Z) + 0.18 * (X ^ X)
# This is the target energy
h2_energy = -1.85727503

# Define the Ansatz
wavefunction = QuantumCircuit(2)
params = ParameterVector('theta', length=8)
it = iter(params)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)
wavefunction.cx(0, 1)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)

# Define the expectation value corresponding to the energy
op = ~StateFn(h2_hamiltonian) @ StateFn(wavefunction)

Now, we can choose whether the `VQE` should use a `Gradient` or `NaturalGradient`, define a `QuantumInstance` to execute the quantum circuits and run the algorithm.

In [24]:
grad = Gradient(grad_method = 'lin_comb')

qi_sv = QuantumInstance(Aer.get_backend('statevector_simulator'),
                                             shots=1,
                                             seed_simulator=2,
                                             seed_transpiler=2)

#Conjugate Gradient algorithm
optimizer = CG(maxiter=50)

# Gradient callable
vqe = VQE(h2_hamiltonian, wavefunction, optimizer=optimizer, gradient=grad)

result = vqe.run(qi_sv)
print('Result:', result['optimal_value'], 'Reference:', h2_energy)

initial -1.0828035790568737
Result: -1.8404998433605153 Reference: -1.85727503


### Compare VQE optimization using the Hessian & Jacobian, Jacobian or none

The `Gradient`, `Hessian`and `QFI` class also have a `gradient_wrapper` method which returns a callable.
This callable takes a set of parameter values and returns the corresponding derivative values.

Using this `gradient_wrapper` we can explore some optimization routines.
In the following we run a `VQE` with different optimizers and compare the results.

In [25]:
from qiskit.aqua.operators import CircuitSampler
from qiskit import BasicAer
def fun(param_values):
    param_dict = dict(zip(params, param_values))
    param_dict = {k: [v] for k, v in param_dict.items()}
    sampler = CircuitSampler(backend=BasicAer.get_backend('qasm_simulator')).convert(op, params=param_dict)
    return sampler.eval()[0]

# Use the gradient wrapper function to build a callable for the Jacobian
jac = Gradient(grad_method = 'param_shift').gradient_wrapper(op, bind_params = params)

# Use the gradient wrapper function to build a callable for the Hessian
hess = Hessian(hess_method = 'param_shift').gradient_wrapper(op, bind_params = params)

# Optimization result using the Hessian and Jacobian
result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, hess=hess, options={'maxiter': 10, 'xtol': 1e-8})
print('VQE using the gradient and the Hessian:', result['fun'], 'Reference:', h2_energy)

# Optimization result using the Jacobian
result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, options={'maxiter': 10, 'xtol': 1e-8})
print('VQE using the gradient:', result['fun'], 'Reference:', h2_energy)

# Gradient-free ptimization result
result = minimize(fun, np.random.rand(len(params)), method='CG', options={'maxiter': 10, 'tol': 1e-8})
print('gradient-free VQE:', result['fun'], 'Reference:', h2_energy)

/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:169: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/_differentiable_functions.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
  self.x = np.atleast_1d(x).astype(float)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:499: ComplexWarning: Casting complex values to real discards the imaginary part
  d1[0, 0] = dc ** 2
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:500: ComplexWarning: Casting complex values to real discards the imaginary part
  d1[0, 1] = -db ** 2
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:501: ComplexWarning: Casting complex values to real discards the imagina

VQE using the gradient and the Hessian: -0.4507812500000001 Reference: -1.85727503


/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:169: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/_differentiable_functions.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
  self.x = np.atleast_1d(x).astype(float)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:499: ComplexWarning: Casting complex values to real discards the imaginary part
  d1[0, 0] = dc ** 2
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:500: ComplexWarning: Casting complex values to real discards the imaginary part
  d1[0, 1] = -db ** 2
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:501: ComplexWarning: Casting complex values to real discards the imagina

VQE using the gradient: -0.3268701320282894 Reference: -1.85727503


/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/ipykernel_launcher.py:24: OptimizeWarning: Unknown solver options: tol
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:519: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:169: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/_differentiable_functions.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
  self.x = np.atleast_1d(x).astype(float)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:519: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Users/ouf/anaconda3/

gradient-free VQE: (-0.2861627984574072+0j) Reference: -1.85727503


### Gibbs state preparation using Variational Quantum Imaginary Time Evolution (VarQITE)

The Gradient Module also facilitates some sophisticated algorithms such as VarQITE for Gibbs state preparation.

For further information on Gibbs state preparation using VarQITE, we would like to refer you to the following work [Variational Quantum Boltzmann Machines](https://arxiv.org/abs/2006.06004).

The core of this Gibbs state preparation routine is a parameter propagation that uses a `NaturalGradient`.

First, the system parameters and operators are defined.

In [26]:
from qiskit.circuit.library import RealAmplitudes

# Set the system temperature...
T = 5

# and the evolution time accordingly.
t =  1/(2*T)

# Define the model Hamiltonian
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I, - 0.5 * I ^ Z^ I^I])

# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.
    
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz    
op = ~StateFn(H) @ CircuitStateFn(ansatz)

# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)

Now, we can set up the `NaturalGradient` and use it to propagate the Ansatz parameters.

In [27]:
# Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
nat_grad = NaturalGradient(grad_method = 'lin_comb', regularization = 'ridge').convert(op, ansatz.ordered_parameters)

param_values = param_values_init
# Propagate the Ansatz parameters step by step according to the explicit Euler method
for step in time_steps:
    param_dict = dict(zip(ansatz.ordered_parameters, param_values))
    nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
    param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))

param_dict_final = dict(zip(ansatz.ordered_parameters, param_values))

gradient [ 0.5  0.3  0.   0.   0.   0.   0.   0.   0.5 -0.2  0.   0.   0.   0.
  0.   0. ]
fubini study [[ 0.25   0.     0.     0.     0.     0.     0.     0.     0.25   0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.25   0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.25  -0.    -0.    -0.    -0.    -0.    -0.    -0.
  -0.    -0.     0.     0.    -0.    -0.   ]
 [ 0.     0.    -0.     0.25  -0.    -0.    -0.    -0.    -0.    -0.
  -0.    -0.     0.     0.    -0.    -0.   ]
 [ 0.     0.    -0.    -0.     0.187  0.187  0.187  0.187 -0.    -0.
  -0.    -0.     0.    -0.     0.187  0.187]
 [ 0.     0.    -0.    -0.     0.187  0.188  0.187  0.187 -0.    -0.
  -0.    -0.     0.    -0.     0.187  0.187]
 [ 0.     0.    -0.    -0.     0.187  0.187  0.187  0.187 -0.    -0.
  -0.    -0.     0.    -0.     0.187  0.187]
 [ 0.     0.    -0.    -0.     0.187  0.187  0.187  0.187 -0.    -0.
  -0.    -0.     0.    -0.   

gradient [ 0.521  0.307  0.001  0.     0.     0.    -0.    -0.     0.51  -0.226
  0.001  0.     0.     0.     0.     0.   ]
fubini study [[ 0.25   0.     0.     0.    -0.    -0.    -0.    -0.     0.25   0.
   0.     0.     0.    -0.    -0.    -0.   ]
 [ 0.     0.25  -0.    -0.     0.     0.     0.     0.    -0.    -0.012
   0.     0.     0.    -0.    -0.    -0.   ]
 [ 0.    -0.     0.25  -0.     0.     0.     0.     0.     0.     0.
   0.001 -0.     0.     0.     0.     0.   ]
 [ 0.    -0.    -0.     0.25   0.     0.     0.     0.     0.     0.
  -0.     0.     0.     0.     0.     0.   ]
 [-0.     0.     0.     0.     0.188  0.187  0.187  0.187  0.    -0.
  -0.    -0.     0.006 -0.008  0.187  0.187]
 [-0.     0.     0.     0.     0.187  0.188  0.187  0.187 -0.    -0.
  -0.    -0.     0.017 -0.008  0.187  0.187]
 [-0.     0.     0.     0.     0.187  0.187  0.188  0.187 -0.     0.
  -0.    -0.     0.017 -0.009  0.187  0.187]
 [-0.     0.     0.     0.     0.187  0.187  0.187  0.188 -0. 

Print the quantum circuit and respective parameters that prepare the approximate Gibbs state

In [28]:
# Print the quantum circuit that integrates the approximate Gibbs state preparation
print(CircuitStateFn(ansatz).assign_parameters(param_dict_final))  
# Print the parameter values which prepare the Gibbs state for the given Ansatz
print(param_values)

CircuitStateFn(
     ┌─────────────────────────┐┌─────────────────────────┐┌───┐»
q_0: ┤ RY(-0.0956420306865894) ├┤ RZ(0.00290696125949448) ├┤ X ├»
     ├─────────────────────────┤├─────────────────────────┤└─┬─┘»
q_1: ┤ RY(-0.0935354349530483) ├┤ RZ(0.00349085198862544) ├──■──»
     └┬────────────────────────┤├─────────────────────────┤     »
q_2: ─┤ RY(0.0136726186940122) ├┤ RZ(0.00345796906027280) ├─────»
      ├────────────────────────┤├─────────────────────────┤     »
q_3: ─┤ RY(0.0138916782785726) ├┤ RZ(0.00345800475791131) ├─────»
      └────────────────────────┘└─────────────────────────┘     »
«     ┌──────────────────────┐┌────────────────────────┐»
«q_0: ┤ RY(1.47920202341531) ├┤ RZ(0.0137522547764177) ├»
«     └────────┬───┬─────────┘└┬──────────────────────┬┘»
«q_1: ─────────┤ X ├───────────┤ RY(1.66619024303395) ├─»
«              └─┬─┘           └────────┬───┬─────────┘ »
«q_2: ───────────■──────────────────────┤ X ├───────────»
«                                       └─